In [16]:
import pandas as pd 
import numpy as np

import xml.etree.ElementTree as ET

In [17]:
spacelist=['','','']

plan=[
['P1',150,150,150,150],
['P2',100,100,100,100],
['P3',100,100,50,50]]
plan=pd.DataFrame(plan,columns=['Pr',1,  2,  3,  4  ])
plan.index=plan['Pr']
plan.drop(columns=['Pr'],inplace=True)

periode=1 #--------------------------------------------------

In [18]:
tree = ET.parse(f'XMLs\\periode_{periode}.xml')#-----------------------------------------------
root = tree.getroot()

waitinglist=list(np.zeros(15)) 

for a in root.find('waitinglistworkstations').findall('workplace'):
    waitinglist[int(a.attrib['id'])-1]+=int(a.attrib['timeneed'])


for a in root.find('waitingliststock').findall('workplace'):
    waitinglist[int(a.attrib['id'])-1]+=int(a.attrib['timeneed'])
    

timeneeddf = pd.DataFrame(spacelist.copy()+waitinglist).T

In [19]:
arbeitsplaetze=15
index_l=[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,26,49,54,29,50,55,30,51,56,31,1,2,3]
u=['K','D','H','K','D','H','K','D','H','K','D','H','KDH','KDH','K','D','H','KDH','K','D','H','K','D','H','K','D','H','K','D','H']

combis=[ 
    [ [6,18,19,20] ],
    [ [5,21,22,23] ],
    [ [5,24],[6,25,26] ],
    [ [6,27],[7,28,29] ],
    [ [0] ],
    [ [3,16,15,14], [2,12]],
    [ [2,16,15,14,17,11,10,9,8,7,6] ],
    [ [1,6,9], [2, 11,10,8,7], [3,16,15,14 ] ],
    [ [3,11,10,8,7,6,9],[2,16,15,14 ] ],
    [ [4,0,1,2,3,4,5]],
    [ [3,0,1,2,3,4,5]],
    [ [3,11,10,8,7,6,9]],
    [ [2,11,10,8,7,6,9]],
    [ [3,12]],
    [ [3,13,17]],
]
def f(x):
    plan_dict={
        'K': plan[periode][0],
        'D': plan[periode][1],
        'H': plan[periode][2],
        'KDH': plan[periode][0]+plan[periode][1]+plan[periode][2]
    }
    return plan_dict[x]

main_df=pd.DataFrame(np.array([u,index_l]).T,columns=['u','NR'])
#main_df['menge']=main_df['menge'].astype(float)
main_df['menge']=main_df['u'].apply(f)


df=pd.DataFrame()
for x in range(1,arbeitsplaetze+1):

    col=np.zeros(len(index_l))

    if x<len(combis)+1:
        for a in combis[x-1]:
            val=a.pop(0)
            
            for y in a:
                col[y]=val

    df[x]=col

for x in range(1,arbeitsplaetze+1):
    main_df[x]=df[x]*main_df['menge']

main_df.index=main_df['NR']


In [20]:
calc_l=spacelist.copy()
for x in range(1,arbeitsplaetze+1):
    calc_l.append(sum(main_df[x]))
#-------------------------1,2,3,4,5,6,7,8,9,10,11,12,13,14
rüstzeit=spacelist.copy()+[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
rüstzeit_p=spacelist.copy()+[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

timeneeddf.columns=main_df.columns
bottom=pd.DataFrame([calc_l,rüstzeit,timeneeddf.to_numpy()[0],rüstzeit_p],columns=main_df.columns)

bottom=bottom.append(bottom.sum().to_frame().T)
bottom.index=['timeneed','rüstzeit','timeneed_p-1','rüstzeit_p-1','total_need']
bottom=bottom.T    

In [21]:
def ueberstunden(x):
    if type(x)==str:
        return '','',''  
    if x < 2400:
        return 0,0,1
    if x > 3500:
        return 0,(x-2400/10),2 if x < 4800 else x-4800,(x-2400/10),2 
    if x > 2400:
        return x-2400,(x-2400)/5,1
    
bottom['Überstunden_total'],bottom['überstunden_tag'],bottom['Schichten']=zip( *bottom['total_need'].apply(ueberstunden) )

production_df=pd.concat([main_df,bottom.T])
production_df.to_csv(f'Production\\prod_{periode}.csv')
production_df

,u,NR,menge,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
4,K,4,150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600.0,450.0,0.0,0.0,0.0,0.0
5,D,5,100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,400.0,300.0,0.0,0.0,0.0,0.0
6,H,6,100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,400.0,300.0,0.0,0.0,0.0,0.0
7,K,7,150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600.0,450.0,0.0,0.0,0.0,0.0
8,D,8,100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,400.0,300.0,0.0,0.0,0.0,0.0
9,H,9,100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,400.0,300.0,0.0,0.0,0.0,0.0
10,K,10,150,0.0,0.0,0.0,0.0,0.0,0.0,300.0,150.0,450.0,0.0,0.0,450.0,300.0,0.0,0.0
11,D,11,100,0.0,0.0,0.0,0.0,0.0,0.0,200.0,200.0,300.0,0.0,0.0,300.0,200.0,0.0,0.0
12,H,12,100,0.0,0.0,0.0,0.0,0.0,0.0,200.0,200.0,300.0,0.0,0.0,300.0,200.0,0.0,0.0
13,K,13,150,0.0,0.0,0.0,0.0,0.0,0.0,300.0,150.0,450.0,0.0,0.0,450.0,300.0,0.0,0.0
